In [1]:
import openai
import os
import pandas as pd
import time
import re

In [2]:
datapath = '../Data/webpage/'
outpath = '../result/'

In [3]:
def save_dataframe(df, save_name, suffix='', outpath=''):
    try:
        # Reset the index
        df_reset = df.reset_index(drop=True)

        # Save as pickle (.pkl)
        pickle_file = f"{outpath}{save_name}{suffix}.pkl"
        df_reset.to_pickle(pickle_file)

        # Save as Excel (.xlsx)
        excel_file = f"{outpath}{save_name}{suffix}.xlsx"
        df_reset.to_excel(excel_file, index=False)

        # Display a message
        print(f"DataFrame saved as {pickle_file} (pickle) and {excel_file} (Excel).")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")

## chatgpt api

In [4]:
# Set your OpenAI API key
openai.api_key = "OpenAI API key"

In [5]:
def generate_prompt(msg,role = 'p'):
    text2 = "\n Please answer the following 3 questions in yes/no. And And combine the answers to the three questions into a list, for example, if the answer is yes,yes,no, then return [yes,yes,no]."
    q1 = "\n Q1: Is this sentence contains greetings?"
    q2 = "\n Q2: Is this sentence contains descriptions of illness or diagnosis? "
    
    if role == 'p':
        text1 = "This is a sentence in a online dialogue of a patient and a doctor, the patient says:"
        q3 = "\n Q3: Is this sentence contains complaints about the doctor?"
    else:
        text1 = "This is a sentence in a online dialogue of a patient and a doctor, the doctor says:"
        q3 = "\n Q3: Is this sentence contains complaints about the patient?"
        
    text3 = "\n The list you return is:"
    prompt = text1 + '"' + msg + '"' + text2 + q1 + q2 + q3 + text3
    return prompt

In [6]:
def labels(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    result = response['choices'][0]['message']['content'].strip()
    #print(result)
    return result

## 读取信息

In [7]:
# 获取目录下所有文件名
all_files = os.listdir(datapath)

# 选择包含 "dialogue" 的 Excel 文件
dia_fls = [file for file in all_files if "dialogue" in file.lower() and file.endswith('.xlsx')]
info_fls = [file for file in all_files if "info" in file.lower() and file.endswith('.xlsx')]

print(len(info_fls))
print(len(dia_fls))

741
746


In [8]:
dfs = []

In [9]:
for i in range(746):
    try:
        dia_fl = dia_fls[i]
        num = dia_fl.split("_")[0]
        dia_file_path = os.path.join(datapath, dia_fl)
        df = pd.read_excel(dia_file_path)
        df.insert(0, 'id', num)
        #print(df)
        
        info_fl = num + "_info.xlsx"
        info_file_path = os.path.join(datapath, info_fl)
        df_info = pd.read_excel(info_file_path)
        doc = df_info.loc[df_info['attribute'] == 'info-text-name', 'value'].values[0]
        doc_id = df_info.loc[df_info['attribute'] == 'doc_id', 'value'].values[0]
        
        df['Sender'] = df['Sender'].apply(lambda x: doc_id if (len(x.split(' '))==2 and x.split(' ')[0]== doc) else x)
        
        dfs.append(df)
        print(i,"successed.")
        
    except:
        pass

result_df = pd.concat(dfs, ignore_index=True)

0 successed.
1 successed.
2 successed.
3 successed.
4 successed.
5 successed.
6 successed.
7 successed.
8 successed.
9 successed.
10 successed.
11 successed.
12 successed.
13 successed.
14 successed.
15 successed.
16 successed.
17 successed.
18 successed.
19 successed.
20 successed.
21 successed.
22 successed.
23 successed.
24 successed.
25 successed.
26 successed.
27 successed.
28 successed.
29 successed.
30 successed.
31 successed.
32 successed.
33 successed.
34 successed.
35 successed.
36 successed.
37 successed.
38 successed.
39 successed.
40 successed.
41 successed.
42 successed.
43 successed.
44 successed.
45 successed.
46 successed.
47 successed.
48 successed.
49 successed.
50 successed.
51 successed.
52 successed.
53 successed.
54 successed.
55 successed.
56 successed.
57 successed.
58 successed.
59 successed.
60 successed.
61 successed.
62 successed.
63 successed.
64 successed.
65 successed.
66 successed.
67 successed.
68 successed.
69 successed.
70 successed.
71 successed.
72

In [10]:
save_dataframe(result_df, "merged_dialogue",outpath)

DataFrame saved as merged_dialogue.pkl (pickle) and merged_dialogue.xlsx (Excel).


## 筛选

In [7]:
df = pd.read_pickle(outpath+"merged_dialogue.pkl")

In [8]:
#df = result_df

In [9]:
df

,id,Time,Sender,Content
0,8897287045,Unknown Time,System,Unknown message type
1,8897287045,11.22,9032334009,你好，平时孩子讲述别的事情时准确吗
2,8897287045,11.22,患者,还可以
3,8897287045,Unknown Time,System,Unknown message type
4,8897287045,11.22,患者,基本都能表达
...,...,...,...,...
12136,8897284906,11.22,266963,问诊建议: 初步诊断：甲状腺肿瘤
12137,8897284906,11.23,患者,请问：我电话问诊的是孙团起教授本人吗？
12138,8897284906,Unknown Time,System,Unknown message type
12139,8897284906,11.23,266963,是本人。


In [10]:
def generate_flag(sender):
    if sender in ['小牛医助', 'System']:
        return -1
    elif sender == '患者':
        return 0
    elif str(sender).isdigit():
        return 1
    else:
        return None  # 可根据实际情况设置其他值或忽略

In [11]:
df['flag'] = df['Sender'].apply(generate_flag)

In [12]:
print(df)

               id          Time      Sender               Content  flag
0      8897287045  Unknown Time      System  Unknown message type  -1.0
1      8897287045         11.22  9032334009     你好，平时孩子讲述别的事情时准确吗   1.0
2      8897287045         11.22          患者                   还可以   0.0
3      8897287045  Unknown Time      System  Unknown message type  -1.0
4      8897287045         11.22          患者                基本都能表达   0.0
...           ...           ...         ...                   ...   ...
12136  8897284906         11.22      266963      问诊建议: 初步诊断：甲状腺肿瘤   1.0
12137  8897284906         11.23          患者   请问：我电话问诊的是孙团起教授本人吗？   0.0
12138  8897284906  Unknown Time      System  Unknown message type  -1.0
12139  8897284906         11.23      266963                  是本人。   1.0
12140  8897284906  Unknown Time      System  Unknown message type  -1.0

[12141 rows x 5 columns]


In [13]:
# 计算每个ID的总行数和每行在ID组中的序号
df['total'] = df.groupby('id')['id'].transform('count')
df['num'] = df.groupby('id').cumcount() + 1
# 计算每个 ID 对应的 'Sender' 不是 '小牛医助' 或 'System' 的行数
df['non_system'] = df.groupby('id')['Sender'].transform(lambda x: x[(x != '小牛医助') & (x != 'System')].count())

# 将 total 列插入为第二列，num 列插入为第三列
df.insert(1, 'total', df.pop('total'))
df.insert(2, 'num', df.pop('num'))
df.insert(3, 'non_system', df.pop('non_system'))


In [14]:
df.head(50)

,id,total,num,non_system,Time,Sender,Content,flag
0,8897287045,40,1,37,Unknown Time,System,Unknown message type,-1.0
1,8897287045,40,2,37,11.22,9032334009,你好，平时孩子讲述别的事情时准确吗,1.0
2,8897287045,40,3,37,11.22,患者,还可以,0.0
3,8897287045,40,4,37,Unknown Time,System,Unknown message type,-1.0
4,8897287045,40,5,37,11.22,患者,基本都能表达,0.0
5,8897287045,40,6,37,11.22,患者,但是有时候也会没准,0.0
6,8897287045,40,7,37,11.22,9032334009,眼前发黑跟体位有关系吗？跟时间有关系吗？,1.0
7,8897287045,40,8,37,11.22,患者,基本是晚上 坐在他的小凳子上喝奶,0.0
8,8897287045,40,9,37,11.22,患者,开始以为是他看了灯 眼前发黑,0.0
9,8897287045,40,10,37,11.22,患者,但是他只有在喝奶的时候会有这种情况,0.0


In [15]:
df = df[df['non_system'] >= 5]

In [16]:
df

,id,total,num,non_system,Time,Sender,Content,flag
0,8897287045,40,1,37,Unknown Time,System,Unknown message type,-1.0
1,8897287045,40,2,37,11.22,9032334009,你好，平时孩子讲述别的事情时准确吗,1.0
2,8897287045,40,3,37,11.22,患者,还可以,0.0
3,8897287045,40,4,37,Unknown Time,System,Unknown message type,-1.0
4,8897287045,40,5,37,11.22,患者,基本都能表达,0.0
...,...,...,...,...,...,...,...,...
12128,8897284439,53,49,43,12.29,患者,一个月的,0.0
12129,8897284439,53,50,43,12.29,System,Unknown message type,-1.0
12130,8897284439,53,51,43,12.29,患者,好的谢谢李医生,0.0
12131,8897284439,53,52,43,12.29,患者,我复查了 肝肾功能和血常规 还需要查尿吗？,0.0


In [18]:
save_dataframe(df, "dialogue_short_del",outpath=outpath)

DataFrame saved as ../result/dialogue_short_del.pkl (pickle) and ../result/dialogue_short_del.xlsx (Excel).


## chatgpt 标注

In [7]:
df = pd.read_pickle(outpath+"dialogue_short_del.pkl")

In [8]:
df

,id,total,num,non_system,Time,Sender,Content,flag
0,8897287045,40,1,37,Unknown Time,System,Unknown message type,-1.0
1,8897287045,40,2,37,11.22,9032334009,你好，平时孩子讲述别的事情时准确吗,1.0
2,8897287045,40,3,37,11.22,患者,还可以,0.0
3,8897287045,40,4,37,Unknown Time,System,Unknown message type,-1.0
4,8897287045,40,5,37,11.22,患者,基本都能表达,0.0
...,...,...,...,...,...,...,...,...
10357,8897284439,53,49,43,12.29,患者,一个月的,0.0
10358,8897284439,53,50,43,12.29,System,Unknown message type,-1.0
10359,8897284439,53,51,43,12.29,患者,好的谢谢李医生,0.0
10360,8897284439,53,52,43,12.29,患者,我复查了 肝肾功能和血常规 还需要查尿吗？,0.0


In [9]:
def fill_columns(row):
    try:
        if row['Content'] == "Unknown message type":
            return ['None', ' None', ' None']
        else:
            if row['flag'] == 0:
                prompt = generate_prompt(row['Content'],role = 'p')
                result = labels(prompt)
                time.sleep(0.1)
            
            elif row['flag'] == 1:
                prompt = generate_prompt(row['Content'],role = 'd') 
                result = labels(prompt)
                time.sleep(0.1)
            
            elif row['flag'] == -1:
                result = "[None, None, None]" 
            
            else:
                result = "[None, None, None]" 
            
            
            #print(result)
            match = re.search(r'\[.*?\]', result)
            result_list = match.group(0).strip('[]').split(',')
            #print(result_list)
            return result_list
    except:
        return ['None', ' None', ' None']
    

In [12]:
batch_size = 100  # 设定每次保存的批量大小
counter = 0  # 计数器，记录处理了多少行数据

start_row = 6202  # 指定开始处理的行数

for index, row in df.iloc[start_row:].iterrows():
    df.loc[index, ['greeting', 'illness/diagnosis', 'complaints']] = fill_columns(row)
    counter += 1

    if counter % batch_size == 0:  # 如果计数器是批量大小的倍数，保存数据
        save_dataframe(df, "example", suffix=f'_{start_row + counter}', outpath=outpath)  # 按照计数器后缀保存
        print(f"Saved data up to row {start_row + counter}.")

# 循环结束后，保存剩余的数据
if counter % batch_size != 0:
    save_dataframe(df, "example", suffix=f'_{start_row + counter}', outpath=outpath)
    print(f"Saved remaining data up to row {start_row + counter}.")


DataFrame saved as ../result/example_6302.pkl (pickle) and ../result/example_6302.xlsx (Excel).
Saved data up to row 6302.
DataFrame saved as ../result/example_6402.pkl (pickle) and ../result/example_6402.xlsx (Excel).
Saved data up to row 6402.
DataFrame saved as ../result/example_6502.pkl (pickle) and ../result/example_6502.xlsx (Excel).
Saved data up to row 6502.
DataFrame saved as ../result/example_6602.pkl (pickle) and ../result/example_6602.xlsx (Excel).
Saved data up to row 6602.
DataFrame saved as ../result/example_6702.pkl (pickle) and ../result/example_6702.xlsx (Excel).
Saved data up to row 6702.
DataFrame saved as ../result/example_6802.pkl (pickle) and ../result/example_6802.xlsx (Excel).
Saved data up to row 6802.
DataFrame saved as ../result/example_6902.pkl (pickle) and ../result/example_6902.xlsx (Excel).
Saved data up to row 6902.
DataFrame saved as ../result/example_7002.pkl (pickle) and ../result/example_7002.xlsx (Excel).
Saved data up to row 7002.
DataFrame saved 

In [14]:
df = pd.read_pickle(outpath+"example_10362.pkl")

In [15]:
# 定义转换函数
def convert_yes_no(value):
    if "y" in value.lower():
        return 1
    elif "n" in value.lower():
        return 0
    else:
        return value

# 遍历 'greeting', 'illness/diagnosis', 'complaints' 三列，应用转换函数
columns_to_convert = ['greeting', 'illness/diagnosis', 'complaints']
df[columns_to_convert] = df[columns_to_convert].applymap(convert_yes_no)

# 显示更新后的 DataFrame
print(df)

               id  total  num  non_system          Time      Sender  \
0      8897287045     40    1          37  Unknown Time      System   
1      8897287045     40    2          37         11.22  9032334009   
2      8897287045     40    3          37         11.22          患者   
3      8897287045     40    4          37  Unknown Time      System   
4      8897287045     40    5          37         11.22          患者   
...           ...    ...  ...         ...           ...         ...   
10357  8897284439     53   49          43         12.29          患者   
10358  8897284439     53   50          43         12.29      System   
10359  8897284439     53   51          43         12.29          患者   
10360  8897284439     53   52          43         12.29          患者   
10361  8897284439     53   53          43         12.29  1207584833   

                     Content  flag greeting  illness/diagnosis complaints  
0       Unknown message type  -1.0        0                  0         

In [16]:
df

,id,total,num,non_system,Time,Sender,Content,flag,greeting,illness/diagnosis,complaints
0,8897287045,40,1,37,Unknown Time,System,Unknown message type,-1.0,0,0,0
1,8897287045,40,2,37,11.22,9032334009,你好，平时孩子讲述别的事情时准确吗,1.0,1,0,0
2,8897287045,40,3,37,11.22,患者,还可以,0.0,0,0,0
3,8897287045,40,4,37,Unknown Time,System,Unknown message type,-1.0,0,0,0
4,8897287045,40,5,37,11.22,患者,基本都能表达,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10357,8897284439,53,49,43,12.29,患者,一个月的,0.0,0,1,0
10358,8897284439,53,50,43,12.29,System,Unknown message type,-1.0,0,0,0
10359,8897284439,53,51,43,12.29,患者,好的谢谢李医生,0.0,1,0,0
10360,8897284439,53,52,43,12.29,患者,我复查了 肝肾功能和血常规 还需要查尿吗？,0.0,0,1,0


In [18]:
save_dataframe(df, outpath+"example", suffix='')

DataFrame saved as ../result/example.pkl (pickle) and ../result/example.xlsx (Excel).


In [27]:
df = pd.read_pickle(outpath+"example.pkl")

In [28]:
id_list = df['id'].tolist()

In [29]:
print(len(list(set(id_list))))

397
